In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
con_dev = ['MLCim', 'MLFaren', 'MLHabibi']

In [4]:
base_dir = '/content/drive/MyDrive/'+con_dev[0]+'/ChaMi-ML/Train'
%cd '$base_dir'

/content/drive/MyDrive/MLCim/ChaMi-ML/Train


In [13]:
# import all package
import data, model, train

In [7]:
!chmod u+r+x ./GetUnzipImg.sh

In [11]:
# get data train and test
train_img, test_img = data.train_test_split()

In [14]:
# set epochs
EPOCHS = 15

## train model
train.train_model(train_img, test_img, EPOCHS)


Epoch 1/15


NameError: ignored